In [61]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model
# Define the model with ResNet50 as the backbone
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import ResNet50


In [62]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Set GPU growth to prevent memory hogging
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  0


In [63]:
# Parameters
IMG_SIZE = 256
BATCH_SIZE = 16
DATA_DIR = "D:/spark-2022-stream-1"       # Base folder
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR = os.path.join(DATA_DIR, "val")
TRAIN_LABEL_FILE = os.path.join(DATA_DIR, "labels/train.csv")
VAL_LABEL_FILE = os.path.join(DATA_DIR, "labels/val.csv")
NUM_CLASSES = 10                            # Number of object classes

# Load bounding box annotations
train_labels_df = pd.read_csv(TRAIN_LABEL_FILE)
val_labels_df = pd.read_csv(VAL_LABEL_FILE)

In [64]:
# Data Generator Class
class DataGenerator(Sequence):
    def __init__(self, dataframe, data_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE, num_classes=NUM_CLASSES):
        self.dataframe = dataframe
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_classes = num_classes
        self.class_names = [
            "Cheops", "lisa_pathfinder", "ObservationSat1", "Proba2", "Proba3", 
            "Proba3ocs", "smart_1", "Soho", "VenusExpress", "XMM Newton"
        ]
        self.class_mapping = {name: idx for idx, name in enumerate(self.class_names)}

    def __len__(self):
        return len(self.dataframe) // self.batch_size

    def __getitem__(self, idx):
        batch_df = self.dataframe.iloc[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        batch_images = []
        batch_bboxes = []
        batch_classes = []

        for _, row in batch_df.iterrows():
            try:
                # Load and preprocess the image
                image_path = os.path.join(self.data_dir, row['class'], row['filename'])
                print(image_path)
                if not os.path.exists(image_path):
                    raise FileNotFoundError(f"Image not found at path: {image_path}")
                image = tf.keras.preprocessing.image.load_img(image_path, target_size=(self.img_size, self.img_size), color_mode='rgb')
                image = tf.keras.preprocessing.image.img_to_array(image) / 255.0
                
                # Scale bounding box coordinates to [0, 1] for the resized image
                xmin, ymin, xmax, ymax = eval(row['bbox'])
                bbox = [xmin / 1024, ymin / 1024, xmax / 1024, ymax / 1024]
                
                # One-hot encode the class label
                class_id = self.class_mapping.get(row['class'], None)
                if class_id is None:
                    raise KeyError(f"Class '{row['class']}' not found in class mapping.")
                class_one_hot = tf.keras.utils.to_categorical(class_id, num_classes=self.num_classes)
                
                batch_images.append(image)
                batch_bboxes.append(bbox)
                batch_classes.append(class_one_hot)
            except (KeyError, FileNotFoundError) as e:
                print(f"Error: {e}. Skipping this row.")

        return np.array(batch_images, dtype=np.float64), {"bbox": np.array(batch_bboxes, dtype=np.float64), "class": np.array(batch_classes, dtype=np.float64)}

# Create training and validation generators
train_gen = DataGenerator(train_labels_df, data_dir=TRAIN_DIR, num_classes=NUM_CLASSES)
val_gen = DataGenerator(val_labels_df, data_dir=VAL_DIR, num_classes=NUM_CLASSES)

In [65]:
def create_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES):
    # Backbone network
    backbone = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    backbone.trainable = False  # Freeze the backbone
    
    # Flatten the backbone output
    x = Flatten()(backbone.output)
    
    # Bounding box regression head
    bbox_head = Dense(1024, activation="relu")(x)
    bbox_head = Dense(512, activation="relu")(bbox_head)
    bbox_output = Dense(4, activation="sigmoid", name="bbox")(bbox_head)  # Bounding box output [xmin, ymin, xmax, ymax]
    
    # Classification head
    class_head = Dense(1024, activation="relu")(x)
    class_head = Dense(512, activation="relu")(class_head)
    class_output = Dense(num_classes, activation="softmax", name="class")(class_head)  # Class output
    
    # Define the model with two outputs
    model = Model(inputs=backbone.input, outputs=[bbox_output, class_output])
    return model

# Initialize and compile the model
model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss={
        "bbox": "mse",  # Loss for bounding box regression
        "class": "categorical_crossentropy"  # Loss for classification
    },
    metrics={"bbox": "mae", "class": "accuracy"}
)


In [66]:

# Train the model with validation data
EPOCHS = 10
model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    verbose=1
)

# Evaluate the model
evaluation = model.evaluate(val_gen)
print("Evaluation Results:", evaluation)

D:/spark-2022-stream-1\train\smart_1\img086351.png
Error: Image not found at path: D:/spark-2022-stream-1\train\smart_1\img086351.png. Skipping this row.
D:/spark-2022-stream-1\train\smart_1\img083734.png
Error: Image not found at path: D:/spark-2022-stream-1\train\smart_1\img083734.png. Skipping this row.
D:/spark-2022-stream-1\train\smart_1\img081317.png
Error: Image not found at path: D:/spark-2022-stream-1\train\smart_1\img081317.png. Skipping this row.
D:/spark-2022-stream-1\train\smart_1\img089140.png
Error: Image not found at path: D:/spark-2022-stream-1\train\smart_1\img089140.png. Skipping this row.
D:/spark-2022-stream-1\train\smart_1\img082379.png
Error: Image not found at path: D:/spark-2022-stream-1\train\smart_1\img082379.png. Skipping this row.
D:/spark-2022-stream-1\train\smart_1\img089707.png
Error: Image not found at path: D:/spark-2022-stream-1\train\smart_1\img089707.png. Skipping this row.
D:/spark-2022-stream-1\train\smart_1\img080704.png
Error: Image not found at

C:\Users\vniko\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("functional_12_1/Cast:0", shape=(None,), dtype=float32). Expected shape (None, 256, 256, 3), but input has incompatible shape (None,)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None,), dtype=float64)
  • training=True
  • mask=None